In [1]:
# imprtamos plotly y vemos la version
import plotly
print(plotly.__version__)


5.17.0


In [2]:
import pandas as pd
# leemos el csv
df = pd.read_csv("risk_factors_cervical_cancer.csv")

In [3]:
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


In [6]:
# importamos dash pandas y modulos necesarios
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Iinicializamos la app
app = dash.Dash(__name__)

# Definimos el layout del app
app.layout = html.Div([
    html.H1("Interactive Dashboard"),
    
    
    # componente dropdown para filtrar por edad
    dcc.Dropdown(
        id='age-dropdown',
        options=[{'label': age, 'value': age} for age in df['Age'].unique()],
        value=df['Age'].unique()[0]
    ),
    
    
    # drowpdowns para seleccionar ejes x y y en el diagrama de dispersion
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[{'label': col, 'value': col} for col in df.columns],
        value='Smokes (years)'
    ),
    dcc.Dropdown(
        id='y-axis-dropdown',
        options=[{'label': col, 'value': col} for col in df.columns],
        value='Smokes (packs/year)'
    ),
    
    # diagrama de dispersion
    html.Div([
        dcc.Graph(id='scatter-plot'),
    ]),
    
    # grafico de barra para numero de parejas sexuales
    html.Div([
        dcc.Graph(id='bar-graph')
    ])
])

# Define a callback to update the scatter plot and bar graph based on the selected Age, x axis, and y axis
@app.callback(
    [Output('scatter-plot', 'figure'), Output('bar-graph', 'figure')],
    [Input('age-dropdown', 'value'), Input('x-axis-dropdown', 'value'), Input('y-axis-dropdown', 'value')]
)
def update_plots(selected_age, x_column, y_column):
    filtered_df = df[df['Age'] == selected_age]
    
    
    scatter_fig = px.scatter(
        filtered_df,
        x=x_column,
        y=y_column,
        color=x_column,
        title=f"Scatter Plot for Age {selected_age}"
    )
    
    
    sorted_df = filtered_df.sort_values(by='Number of sexual partners', ascending=True)
    bar_fig = px.bar(
        sorted_df,
        x='Number of sexual partners',
        title=f"Bar Graph for Number of Sexual Partners for Age {selected_age}"
    )
    # retornamos ambas figuras
    return scatter_fig, bar_fig

if __name__ == '__main__':
    app.run_server(debug=True)
